```kotlin
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            FlowBasicsTheme {
                FlowDemoScreen()
            }
        }
    }
}

/**
 * Ta funkcja definiuje "zimny" strumień.
 * Jest jak przepis - sam w sobie nic nie robi.
 * Kod wewnątrz bloku flow { ... } wykona się dopiero po wywołaniu .collect()
 */
fun numberStream(logs: SnapshotStateList<String>): Flow<Int> = flow {
    val timeFormat = SimpleDateFormat("HH:mm:ss", Locale.getDefault())
    logs.add("[${timeFormat.format(Date())}] Strumień wystartował! 🚀")
    for (i in 1..5) {
        delay(1000)
        logs.add("  - Emituję wartość: $i")
        emit(i)
    }
    logs.add("[${timeFormat.format(Date())}] Strumień zakończył pracę.")
}

@Composable
fun FlowDemoScreen() {
    val scope = rememberCoroutineScope()
    val logs = remember { mutableStateListOf<String>() }
    var latestValue by remember { mutableStateOf(0) }
    val listState = rememberLazyListState()

    // Automatyczne scrollowanie listy logów na dół
    LaunchedEffect(logs.size) {
        if (logs.isNotEmpty()) {
            listState.animateScrollToItem(logs.size - 1)
        }
    }

    Column(
        modifier = Modifier.fillMaxSize().padding(16.dp),
        horizontalAlignment = Alignment.CenterHorizontally
    ) {
        Text(
            "Demonstracja 'Zimnego' Flow",
            style = MaterialTheme.typography.headlineMedium,
            textAlign = TextAlign.Center
        )
        Spacer(Modifier.height(20.dp))

        Card(modifier = Modifier.padding(16.dp)) {
            Text(
                text = "$latestValue",
                fontSize = 52.sp,
                fontWeight = androidx.compose.ui.text.font.FontWeight.Bold,
                modifier = Modifier.padding(horizontal = 32.dp, vertical = 16.dp)
            )
        }

        Button(
            onClick = {
                // Każde kliknięcie uruchamia NOWĄ korutynę,
                // która tworzy NOWĄ subskrypcję na strumień.
                scope.launch {
                    logs.add("--- NOWA SUBSKRYPCJA ---")
                    numberStream(logs).collect { number ->
                        latestValue = number
                    }
                }
            },
            modifier = Modifier.fillMaxWidth()
        ) {
            Text("Subskrybuj Strumień (Start od nowa)")
        }

        Spacer(Modifier.height(16.dp))

        // Wyświetlanie logów
        LazyColumn(
            state = listState,
            modifier = Modifier.fillMaxSize().padding(8.dp),
        ) {
            items(logs) { log ->
                Text(log, fontFamily = FontFamily.Monospace, modifier = Modifier.padding(vertical = 2.dp))
            }
        }
    }
}
```